In [36]:
import os
from dotenv import load_dotenv
from supabase import create_client, Client

import pandas as pd

load_dotenv()
url: str = os.getenv("SUPABASE_URL")
key: str = os.getenv("SUPABASE_KEY")
supabase: Client = create_client(url, key)

def get_staff_skill(staff_id: int):
    staff_skill_id = supabase.from_('staff_skill').select("skill_id").eq('staff_id', staff_id).execute().data
    staff_skill_id = [skill['skill_id'] for skill in staff_skill_id]

    staff_skill = supabase.from_('skill').select("*").in_('skill_id', staff_skill_id).execute().data
    return staff_skill

def get_role_skill(role_id: int):
    role_skill_id = supabase.from_('role_skill').select("skill_id").eq('role_id', role_id).execute().data
    role_skill_id = [skill['skill_id'] for skill in role_skill_id]

    role_skill = supabase.from_('skill').select("*").in_('skill_id', role_skill_id).execute().data
    return role_skill

def get_staff_role_skill(staff_id: int, role_id: int):
    # Get the list of skill_id for the staff and role
    staff_skill = pd.DataFrame(get_staff_skill(staff_id))
    role_skill = pd.DataFrame(get_role_skill(role_id))
    
    # Add a 'qualified' field to the staff_skill list
    role_skill['qualified'] = role_skill.skill_id.isin(staff_skill.skill_id)

    # Sort the staff_skill list based on the 'qualified' field
    return role_skill.sort_values(by=['qualified', 'skill_name'], ascending=[False, True])

get_staff_role_skill(2, 1)

2023-09-27 12:04:46,492:INFO - HTTP Request: GET https://wbsagjngbxrrzfktkvtt.supabase.co/rest/v1/staff_skill?select=skill_id&staff_id=eq.2 "HTTP/1.1 200 OK"
2023-09-27 12:04:46,571:INFO - HTTP Request: GET https://wbsagjngbxrrzfktkvtt.supabase.co/rest/v1/skill?select=%2A&skill_id=in.%281%2C3%2C15%2C2%2C7%2C8%2C11%29 "HTTP/1.1 200 OK"
2023-09-27 12:04:47,326:INFO - HTTP Request: GET https://wbsagjngbxrrzfktkvtt.supabase.co/rest/v1/role_skill?select=skill_id&role_id=eq.1 "HTTP/1.1 200 OK"
2023-09-27 12:04:48,271:INFO - HTTP Request: GET https://wbsagjngbxrrzfktkvtt.supabase.co/rest/v1/skill?select=%2A&skill_id=in.%282%2C7%2C8%2C9%2C11%2C18%29 "HTTP/1.1 200 OK"


,skill_id,skill_name,skill_desc,qualified
1,7,Django,Python web framework,True
2,8,Java Class-based,object-oriented programming language,True
4,11,PostgreSQL,Open-source relational database management system,True
0,2,Product Management,Manage product,True
5,18,PHP,Server-side scripting language,False
3,9,Spring,Java application framework,False


In [28]:
staff_skill = pd.DataFrame(get_staff_skill(2))
role_skill = pd.DataFrame(get_role_skill(1))

role_skill['qualified'] = role_skill.skill_id.isin(staff_skill.skill_id)

role_skill

2023-09-27 11:25:37,347:INFO - HTTP Request: GET https://wbsagjngbxrrzfktkvtt.supabase.co/rest/v1/staff_skill?select=skill_id&staff_id=eq.2 "HTTP/1.1 200 OK"
2023-09-27 11:25:37,398:INFO - HTTP Request: GET https://wbsagjngbxrrzfktkvtt.supabase.co/rest/v1/skill?select=%2A&skill_id=in.%281%2C3%2C15%2C2%2C7%2C8%2C11%29 "HTTP/1.1 200 OK"
2023-09-27 11:25:37,466:INFO - HTTP Request: GET https://wbsagjngbxrrzfktkvtt.supabase.co/rest/v1/role_skill?select=skill_id&role_id=eq.1 "HTTP/1.1 200 OK"
2023-09-27 11:25:37,536:INFO - HTTP Request: GET https://wbsagjngbxrrzfktkvtt.supabase.co/rest/v1/skill?select=%2A&skill_id=in.%282%2C7%2C8%2C9%2C11%2C18%29 "HTTP/1.1 200 OK"


,skill_id,skill_name,skill_desc,qualified
0,2,Product Management,Manage product,True
1,7,Django,Python web framework,True
2,8,Java Class-based,object-oriented programming language,True
3,9,Spring,Java application framework,False
4,11,PostgreSQL,Open-source relational database management system,True
5,18,PHP,Server-side scripting language,False


In [34]:
role_skill.sort_values(by=['qualified', 'skill_name'], ascending=[False, True])

,skill_id,skill_name,skill_desc,qualified
1,7,Django,Python web framework,True
2,8,Java Class-based,object-oriented programming language,True
4,11,PostgreSQL,Open-source relational database management system,True
0,2,Product Management,Manage product,True
5,18,PHP,Server-side scripting language,False
3,9,Spring,Java application framework,False


In [30]:
staff_skill

,skill_id,skill_name,skill_desc
0,1,Python,Use Python to code
1,2,Product Management,Manage product
2,3,Scrum,Use scrum methodology
3,7,Django,Python web framework
4,8,Java Class-based,object-oriented programming language
5,11,PostgreSQL,Open-source relational database management system
6,15,MongoDB,NoSQL database program
